In [ ]:
%pip install pandas beautifulsoup4 selenium

In [303]:
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.edge.options import Options
from selenium.webdriver.common.by import By

In [304]:
url = "https://leagueoflegends.fandom.com/wiki/List_of_champions"
option = Options()
option.headless= True
driver = webdriver.Edge(options=option)

In [305]:
driver.get(url)
element = driver.find_element(By.XPATH,"//*[@id='mw-content-text']/div[1]/table")
htmlContent = element.get_attribute("outerHTML")

soup = BeautifulSoup(htmlContent,'html.parser')
leagueTable = soup.find('table')
driver.quit()

Note:
>`#1.`Correction required in other places marked if data here as changed:

In [306]:
listChampions = []

for champions in leagueTable.find_all('tbody'):
    champion = champions.find_all('tr')
    for row in champion:
        championName = row.find('span', class_='inline-image label-only champion-icon tooltips-init-complete')['data-champion'] #1.
        championClass = row.find('span', class_='glossary tooltips-init-complete')['data-tip']
        championReleaseDate = row.find_all('td')[2].text 
        championLastChanged = row.find_all('td')[3].text
        championBlueEssence = row.find_all('td')[4].text
        championRP = row.find_all('td')[5].text
        championImgUrl = row.find('img')['data-src']

        listChampions.append([championName, championClass, championReleaseDate,
                             championLastChanged, championBlueEssence, championRP, championImgUrl]) #1.


In [307]:
def remove_breakline(text):
    return text.replace("\n", "")

In [308]:
df = pd.DataFrame (listChampions, columns=['name','class','releaseDate','lastChanged','blueEssence','riotPoints','imgUrl']) #1.
df = df.applymap(remove_breakline)

In [309]:
df.to_json('AllChampionsList.json',orient='records', lines = False)

driver.quit()